In [29]:
'''
Pipeline issue to be resolved...
Exact same code here but it works
Run every week if needed

'''
import os
import pandas as pd
import glob
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import urllib
from utils.marketinsights import process_tables, get_city_mapping, get_contact_information, get_embeddings, get_industry, get_intersection, get_phone_mapping, label_country_by_city, label_country_by_phone, final_country, convert_to_datetime, AutoModel, AutoTokenizer

model_name = 'nomic-ai/nomic-embed-text-v1'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

selection = 'marketinsights'
directory = "./utils/data/Scraped News/"
file_pattern = f"{selection}_data_*.csv"
files = glob.glob(os.path.join(directory, file_pattern))
print(files)
latest_file = max(files, key=os.path.getctime)
print(latest_file)

df = pd.read_csv(latest_file)

df_temp = df['tables'].apply(lambda x: pd.Series(process_tables(x)))
df_temp.columns = ['Executives', 'Shareholders']
df[['Executives', 'Shareholders']] = df_temp

print('Labelling Country and Industry')

try:
    existing = pd.read_csv('utils/data/Scrape/phoneextensions.csv')
except:
    existing = pd.DataFrame()
phone_storage = get_phone_mapping(existing)
city_storage = get_city_mapping()
df['Industry'] = df['raw'].apply(get_industry)
df['Contact Information'] = df['raw'].apply(get_contact_information)
df['Country_phone'] = df['Contact Information'].apply(lambda x: label_country_by_phone(x, phone_storage))
df['Country_city'] = df['Contact Information'].apply(lambda x: label_country_by_city(x, city_storage))
df['Country_candidates'] = df.apply(lambda x: get_intersection(x.Country_phone, x.Country_city), axis = 1)
df['Country'] = df.apply(lambda x: final_country(x['Contact Information'], x.Country_candidates, tokenizer, model), axis = 1)

# Apply the conversion functions
df['Time'] = df['date'].apply(convert_to_datetime)
df.drop(['date'], axis = 1)
df.to_csv('utils/data/Scraped News/marketinsights_data_2024-07-11.csv')

/Users/launchpad/miniforge3/envs/argus/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
<All keys matched successfully>


['./utils/data/Scraped News/marketinsights_data_temp.csv']
./utils/data/Scraped News/marketinsights_data_temp.csv
Labelling Country and Industry


In [26]:
'''
Trying out sentiment scoring for article content
'''

from transformers import BertForSequenceClassification, BertTokenizer
import torch

test_content = df.iloc[5]['Article content']
print(test_content)

# initialize our model and tokenizer
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')


def get_sentiment(text):
    tokens = tokenizer.encode_plus(text, add_special_tokens=False)
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    # define our starting position (0) and window size (number of tokens in each chunk)
    start = 0
    window_size = 512
    
    # initialize probabilities list
    probs_list = []
    
    start = 0
    window_size = 510  # we take 2 off here so that we can fit in our [CLS] and [SEP] tokens
    
    loop = True
    
    while loop:
        end = start + window_size
        if end >= total_len:
            loop = False
            end = total_len
        # (1) extract window from input_ids and attention_mask
        input_ids_chunk = input_ids[start:end]
        attention_mask_chunk = attention_mask[start:end]
        # (2) add [CLS] and [SEP]
        input_ids_chunk = [101] + input_ids_chunk + [102]
        attention_mask_chunk = [1] + attention_mask_chunk + [1]
        # (3) add padding upto window_size + 2 (512) tokens
        input_ids_chunk += [0] * (window_size - len(input_ids_chunk) + 2)
        attention_mask_chunk += [0] * (window_size - len(attention_mask_chunk) + 2)
        # (4) format into PyTorch tensors dictionary
        input_dict = {
            'input_ids': torch.Tensor([input_ids_chunk]).long(),
            'attention_mask': torch.Tensor([attention_mask_chunk]).int()
        }
        # (5) make logits prediction
        outputs = model(**input_dict)
        # (6) calculate softmax and append to list
        probs = torch.nn.functional.softmax(outputs[0], dim=-1)
        probs_list.append(probs)
    
        start = end

    stacks = torch.stack(probs_list)
    shape = stacks.shape
    with torch.no_grad():
        # we must include our stacks operation in here too
        stacks = torch.stack(probs_list)
        # now resize
        stacks = stacks.resize_(stacks.shape[0], stacks.shape[2])
        # finally, we can calculate the mean value for each sentiment class
        mean = stacks.mean(dim=0)
    winner = torch.argmax(mean).item()
    result = ['Positive', 'Negative', 'Neutral'][winner]
    return result

df['Sentiment'] = df['Article content'].apply(get_sentiment)
for i, row in df.iterrows():
    print('Content: \n')
    print(row['Article content'])
    print('\nSentiment: \n')
    print(row['Sentiment'])
    print('**********')

In [48]:
'''
Trying out labelling missing executives from company website
'''

from serpapi import GoogleSearch
import pandas as pd
import streamlit as st
import requests
import re

def search_company_management(company_name, api_key):
    # Define common terms used for management page
    management_terms = {
        "Team", "Executives", "Leadership", 
        "Our Team", "Leaders", 
        "Directors", "Management", 
        "Executive", "Board"}
    
    # Construct query with various combinations
    queries = f"{company_name} company AND {' OR '.join(management_terms)}"
    
    params = {
        "q": queries,
        "engine": "google",
        "google_domain": "google.com",
        "num": 10,
        "api_key": api_key,
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    return results
def extract_management_info(text):
    # Define common position titles
    position_titles = [
        "CEO", "Chief Executive Officer", "CFO", "Chief Financial Officer", "COO", 
        "Chief Operating Officer", "CTO", "Chief Technology Officer", "CMO", 
        "Chief Marketing Officer", "Manager", "Director", "Vice President", 
        "VP", "President", "Chairman", "Founder", "Owner", "Partner", "Head"
    ]
    
    # Create a regex pattern to find names and positions
    # This pattern assumes that names are capitalized words (with optional middle names/initials)
    name_pattern = r'([A-Z][a-z]+(?: [A-Z][a-z]*)*)'
    position_pattern = r'(' + '|'.join(position_titles) + r')'
    combined_pattern = re.compile(rf'{name_pattern}\s*,?\s*{position_pattern}|{position_pattern}\s*,?\s*{name_pattern}', re.IGNORECASE)
    
    # Find all matches in the text
    matches = combined_pattern.findall(text)
    
    # Clean up matches
    management_info = []
    for match in matches:
        # Filter out empty strings and None
        filtered_match = [m for m in match if m]
        if len(filtered_match) == 2:
            if filtered_match[1] in position_titles:
                management_info.append((filtered_match[0].strip(), filtered_match[1].strip()))
            else:
                management_info.append((filtered_match[1].strip(), filtered_match[0].strip()))
    
    return management_info


In [39]:
# Example usage
api_key = st.secrets['serp_api_key']
company_name = 'Snapdocs'
search_results = search_company_management(company_name, api_key)
print(search_results)
temp = []
search_results.keys()
for dictionary in search_results['organic_results']:
    temp.append(dictionary['link'])



In [49]:
storage = {}
for link in temp:
    headerx = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
        }
    # Send a GET request to the URL
    try:
        response = requests.get(link, headers = headerx)
        response.raise_for_status()  # Raise an exception if the request was unsuccessful
        # Parse the HTML content of the page with BeautifulSoup
        html_content = urllib.parse.unquote(response.text)
        soup = BeautifulSoup(html_content, 'html.parser')
        # Extract and clean text content
        text_content = soup.get_text(separator=' ', strip=False)
        
        # Extract management information
        management_info = extract_management_info(text_content)
        
        # Print the extracted management information
        for name, position in management_info:
            if name not in storage.keys():
                storage[name] = {position: 1}
            else:
                if position not in storage[name].keys():
                    storage[name] = {position: 1}
                else:
                    storage[name][position] += 1
    except:
        print(f"Unable to scrape {link}")

Unable to scrape https://ir.tesla.com/corporate
Unable to scrape https://www.wsj.com/market-data/quotes/TSLA/company-people
Unable to scrape https://ir.tesla.com/contact-us
Name: CSR, Position: Partner
Name: cto, Position: Public Se
Name: cto, Position: Actionable se
Name: cto, Position: Fa
Name: executives, Position: CEO
Name: Elon Musk, Position: Chief Executive Officer
Name: cto, Position: Dire
Name: Elon Musk has been the, Position: Chief Executive Officer
Name: cto, Position: and Dire
Name: and was, Position: Chairman
Name: he also serves as the Chief Executive Officer, Position: Chief Technology Officer
Name: and, Position: Chairman
Name: of Twitter Inc since, Position: Chief Executive Officer
Name: cto, Position: and Dire
Name: Senior Management, Position: Chief Financial Officer
Name: Vaibhav Taneja has been the, Position: Chief Financial Officer
Name: Denholm, Position: Chairman
Name: Denholm has been the, Position: Chairman
Name: cto, Position: she served as the Chief Financi